In [2]:
!pip install --quiet --upgrade google-cloud-logging google_cloud_firestore google_cloud_aiplatform langchain langchain-google-vertexai langchain_community langchain_experimental pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [1]:
import vertexai
import logging
import google.cloud.logging
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel

import pickle
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

In [2]:
PROJECT_ID = "qwiklabs-gcp-03-1144253aecd8"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
embedding_model = VertexAIEmbeddings(model_name="text-embedding-005")

/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [4]:
!gcloud storage cp gs://qwiklabs-gcp-03-1144253aecd8-bucket/embeddings.json .

Copying gs://qwiklabs-gcp-03-1144253aecd8-bucket/embeddings.json to file://./embeddings.json


In [5]:
import json

with open("embeddings.json", "r") as f:
    precomputed = json.load(f)

chunks = []
embeddings = []

for item in precomputed:
    chunks.append(item["content"]) # Extract the text chunk
    embeddings.append(item["embedding"]) # Extract the embedding vector

In [6]:
from google.cloud import firestore

# Initialize Firestore client
db = firestore.Client(project=PROJECT_ID)
collection = db.collection("food_safety_chunks")

# Store each embedding and chunk
for i, (embedding, chunk) in enumerate(zip(embeddings, chunks)):
    doc = {
        "embedding": embedding,
        "chunk": chunk
    }
    collection.document(f"chunk_{i}").set(doc)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def query_firestore(query_text):
    model = TextEmbeddingModel.from_pretrained("text-embedding-005")
    query_embedding = model.get_embeddings([query_text])[0].values

    docs = collection.stream()
    chunks_data = [(doc.to_dict()["chunk"], doc.to_dict()["embedding"]) for doc in docs]

    sims = [(chunk, cosine_similarity([query_embedding], [emb])[0][0]) for chunk, emb in chunks_data]
    top_chunks = sorted(sims, key=lambda x: x[1], reverse=True)[:3]

    return "\n".join([chunk for chunk, _ in top_chunks])

In [8]:
query = "What should you do if food is left out overnight?"
relevant_text = query_firestore(query)

chat_model = GenerativeModel("gemini-2.0-flash")  # Instantiate GenerativeModel directly
chat = chat_model.start_chat()

response = chat.send_message(
    f"Use the following to answer the question:\n\n{relevant_text}\n\nQuestion: {query}"
)
print(response.text)

/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Based on the provided information about food safety and proper storage, if food is left out overnight, you should **discard it**. The information emphasizes the importance of proper storage for safety and quality, implying that leaving food out at room temperature for an extended period (like overnight) compromises its safety and could lead to foodborne illnesses.

